<h1 style = "font-size:25px">Nexus site scraping</h1>
<h2 style="font-size:15px">  Author: Jacob Massengill, et. al. (add your name if you edit this)</h2>

<p>This notebook defines classes and functions that will be used to scrape the Nexus skyrim site
</p>

In [47]:
#define a timer which will be used to 
#regulate the speed at which pages will be
#retrieved

#pages retrieved/sec = (1/self.mtime)
import time
class StopWatch:
    def __init__(self, time):
        self.mtime = time
    def start(self):
        ctime = time.time()+self.mtime
        while(ctime > time.time()):
            pass

In [6]:
#demo of stop_watch
timer = StopWatch(1)
for i in reversed(range(10)):
    timer.start()
    print (i+1, end=" ")

10 9 8 7 6 5 4 3 2 1 

In [4]:
#defines a function for getting the urls for each
#nexus skyrim category
import requests
from bs4 import BeautifulSoup

#returns a list of category urls from the nexus skyrim site
def get_nexus_cat_urls(url='http://www.nexusmods.com/skyrim/mods/categories/?'):

    #get categories page
    main_page = requests.get(url)
    main_soup = BeautifulSoup(main_page.text, 'html')

    #get category urls from categories page
    #they will be the initialized stack for the webcrawler
    cat_list = main_soup.find('ul', class_="category-list")
    cat_urls = []
    for link in cat_list.find_all('a', href=True):
        cat_urls.append(link['href'])
    
    #remove the top element, it is a site statistic href
    cat_urls.pop()
    return cat_urls

In [ ]:
#demo for get_nexus_cat_urls
cat_urls = get_nexus_cat_urls()
for link in sorted(cat_urls):
    print (link)

<p><h1 style="font-size:12px">In the cells below, need to define:</h1><br>
   1) a class for holding statistics about each mod <br>
   2) function for getting those statistics using cat_urls <br>
   3) function for exploring each mod's particular page <br>
</p>

In [32]:
#read some url pages, study what else can be added

#container class for information on mod
class ModBlock:
    def __init__(self):
        self.url = ''
        self.likes = 0
        self.downloads = 0
        self.name = ''
        self.des = ''
        self.created = ''
        self.update = ''
        self.creator= ''
    def __init__(self,html):
        self.from_html(html)
    def print_mod(self):
        print('{0}: {1}'.format('url', self.url))
        print('{0}: {1}'.format('likes', self.likes))
        print('{0}: {1}'.format('downloads', self.downloads))
        print('{0}: {1}'.format('name', self.name))
        print('{0}: {1}'.format('description', self.des))
        print('{0}: {1}'.format('created', self.created))
        print('{0}: {1}'.format('updated', self.update))
        print('{0}: {1}'.format('created by', self.creator))
    #should be an entry from get_nexus_mods
    def from_html(self, html):
        self.url = html.find('a', class_='image bubble-open pb-hover pb-left pb-ajax pb-forceclose', href=True)['href']
        self.likes = html.find('span', class_='likes').text
        self.downloads = html.find('span', class_='downloads').text
        self.name = html.find('a', class_='title').text
        self.des = html.find('div', class_=None).text
        self.created = html.find('div', class_='category-file-hover-released').text
        self.update = html.find('div', class_='category-file-hover-updated').text
        self.creator = html.find('a', class_='user').text
        

In [54]:
#gets a list of the nexus mods at url
#currently just gets the html. 
#change this to put things into a class
def get_nexus_mods(url):
    page = requests.get(url)
    
    if page.status_code != 200:
        return None
    
    soup = BeautifulSoup(page.text, 'html')

    blockList = soup.find('ul', class_="block-list")
    popboxes = blockList.find_all('li', class_='popbox')
    
    return popboxes

In [28]:
#demo of get_nexus_mods
cat_urls = get_nexus_cat_urls()
mods = get_nexus_mods(cat_urls[0])
print(mods[3])

<li class="popbox">
				<a class="image bubble-open pb-hover pb-left pb-ajax pb-forceclose" href="http://www.nexusmods.com/skyrim/mods/70322/">
					<p class="stats">
						<span class="likes">11</span>
						<span class="downloads">94</span> 
					</p>
					<span class="stats-overlay">
					</span>
					<div class="image-overlay" style="display: none;">
						<p></p>
					</div>
					<center>
						<img alt="Swampwater Cottage" src="http://static-2.nexusmods.com/15/mods/110/images/thumbnails/70322-6-1444011050.jpg"/>
					</center>
				</a>
				<div class="bubble-collapse">
					<div class="bubble-box category-file-hover-desc" style="display: none; left: 192px; bottom: -56px; z-index:99;">
						<div class="bubble-arrow arrowleft" style="top: 94.5px;"></div>
						<div class="bubble-arrow-border arrowleft-border" style="top: 94.5px;"></div>
						<div class="heading-wrap">Swampwater Cottage</div>
						<div>
												</div>
						<div class="category-file-hover-released">Released: 

In [36]:
#demo of ModBlock. get a list of modBlocks from single nexus mod page
modList = []
for e in mods:
    modList.append(ModBlock(e))
modList[0].print_mod()

url: http://www.nexusmods.com/skyrim/mods/70414/
likes: 13
downloads: 204
name: HauntedLady1.1
description: 
						My sixth "full service" player home with room for several followers. Has lots of storage an armory and dungeon pool.  Great mod for starting a new game or any level.  Takes very little memory. No DLC required. 						
created: Released: 08/10/2015 - 09:26PM
updated: Updated: 10/10/2015 - 06:32PM
created by: Whitemorn


In [57]:
#returns a list of modblocks from pages start to end (inclusive)
#grabs pages at a rate of rr/sec

def get_nexus_mods_from_pages(start = 1, end = 1, rr = 1):
    timer = StopWatch(rr)
    mods = []
    for i in range(start, end+1):
        timer.start()
        url = 'http://www.nexusmods.com/skyrim/mods/searchresults/?src_order=3&src_sort=0&src_view=1&src_tab=1&src_language=0&page='+str(i)+'&pUp=1'
        
        modList = get_nexus_mods(url)
        if modList == None:
            print('no mods on page {0}\nare you sure your\'e in range?'.format(url))
        for e in modList:
            mods.append(ModBlock(e))
    return mods

In [59]:
mods = get_nexus_mods_from_pages(end=3)
mods[0].print_mod()

url: http://www.nexusmods.com/skyrim/mods/607/
likes: 130,790
downloads: 17,055,974
name: Skyrim HD - 2K Texture...
description: 
						The most downloaded high resolution texture mod for skyrim.						
created: Released: 19/11/2011 - 01:03AM
updated: Updated: 04/10/2015 - 12:49PM
created by: NebuLa1


In [60]:
print(len(mods))

90
